# TCC Augusto e Ícaro
## Modelo de automatização das Heurísticas de Nielsen para comentários em reviews de Apps

* Versão 1.0
* Bibliotecas utilizadas: pandas, numpy, texthero, ntlk e corpus do ntlk em português
* Dataset utilizado: dataset_v4.csv
* Data: 22/07/2020

### Objetivos:

[x] Pre processamento detalhado

[x] Pipeline de pre-processamento

[x] Classificador baseado em ocorrência

In [108]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install texthero 
!{sys.executable} -m pip install nltk 

In [1]:
import pandas as pd
import texthero as hero
import nltk
import numpy as np
# nltk.download()

## Pré-processamento

Nessa versão iremos testar apenas com textos em ingles, sem adicionar os pesos de sentimentos e heurísticas de Nielsen. Queremos treinar o modelo para classificar apenas em usabilidade ou não.

In [2]:
cols = ['Text', 'is_usability', 'is_classified']
df = pd.read_csv("reviews_v4.csv", index_col=False, usecols=cols)
df = df.rename(columns={'Text': 'text'})
df.head()

,text,is_usability,is_classified
0,Negócio de reconhecimento facialnão funciona. ...,True,True
1,Um lixo!!! 9 tentativas de econhecimento facia...,False,True
2,Horrível! Pior que FBI! Se fosse pra receberem...,True,True
3,Meio difícil fazer sem óculos mais deu certo,True,True
4,"Não serve pra nada , não dá para acessar,péssimo.",True,True


In [3]:
df['is_usability'].value_counts()

False    755
True     175
Name: is_usability, dtype: int64

In [4]:
df['is_classified'].value_counts()

False    548
True     381
Name: is_classified, dtype: int64

### Limpando dados

O pipeline padrão remove dígitos, pontuação, remove diacritics, stopwords em inglês e whitespace

In [5]:
pipeline = hero.preprocessing.get_default_pipeline()
selected_functions_indexes = [0, 1, 2, 5, 6]
pipeline = [pipeline[i] for i in selected_functions_indexes]
df['text'] = hero.preprocessing.clean(df['text'])
df['text']

0      negocio de reconhecimento facialnao funciona n...
1      um lixo tentativas de econhecimento facial sem...
2      horrivel pior que fbi se fosse pra receberem a...
3           meio dificil fazer sem oculos mais deu certo
4         nao serve pra nada nao da para acessar pessimo
                             ...                        
925    como eu faco pra entrar na minha conta enem pe...
926    sim facilita muito em varios servicos e mais i...
927    nao consigo acessar meu auxilio como faco pra ...
928                                  seguro hiper seguro
929    otimo porem e bom implementar novas funcionali...
Name: text, Length: 930, dtype: object

### Stopwords

Removendo stopwords em português com o corpus em portugês do NTLK

In [6]:
from nltk.corpus import stopwords
stopwords = stopwords.words('portuguese')
df['text'] = hero.remove_stopwords(df['text'], stopwords=stopwords)
df['text'][0]

'negocio  reconhecimento facialnao funciona nao  pra ficar dia todo nisso nao arrumem   favor'

### Stemização

In [7]:
df['text'] = hero.stem(df['text'], language='portuguese')
df['text']
df['text'][0]

'negoci reconhec facialna funcion nao pra fic dia tod niss nao arrum favor'

### Tokenização

In [8]:
df['text'] = hero.tokenize(df['text'])
df.head()

,text,is_usability,is_classified
0,"[negoci, reconhec, facialna, funcion, nao, pra...",True,True
1,"[lix, tentat, econhec, facial, exit, dur, depe...",False,True
2,"[horrivel, pior, fbi, pra, receb, algo, gent, ...",True,True
3,"[mei, dificil, faz, ocul, deu, cert]",True,True
4,"[nao, serv, pra, nad, nao, acess, pessim]",True,True


In [9]:
df['class_name'] = df.apply(lambda row: 'usability' if row['is_usability'] else 'not_usability', axis=1)

In [10]:
df = df.drop('is_usability', 1)
df = df.to_dict('records')
df[0]

{'text': ['negoci',
  'reconhec',
  'facialna',
  'funcion',
  'nao',
  'pra',
  'fic',
  'dia',
  'tod',
  'niss',
  'nao',
  'arrum',
  'favor'],
 'is_classified': True,
 'class_name': 'usability'}

## NLP

Nessa parte, iremos treinar e utilizar o modelo

Função para aprendizado dos dados: 

In [11]:
def learning(training_data):
    corpus_words = {}
    for data in training_data: 
        class_name = data['class_name']
        frase = data['text']
        if class_name not in list(corpus_words.keys()):
            corpus_words[class_name] = {}
        for word in frase:
            if word not in list(corpus_words[class_name].keys()):
                corpus_words[class_name][word] = 1
            else:
                corpus_words[class_name][word] += 1
    return corpus_words

Função para calcular scores

In [12]:
def classificate(corpus, sentence):
    def calculate_class_score(corpus_words, sentence, class_name):
        score = 0 
        for word in sentence:
            if word in corpus_words[class_name]:
                score += corpus_words[class_name][word]
        return score
    classifications = []
    for class_name in corpus.keys():
        classifications.append({'class_name': class_name, 'score': calculate_class_score(corpus, sentence, class_name)})    
    return classifications

Função para normalizar os scores

In [13]:
def normalize_scores(classification):
    total_score = sum(score['score'] for score in classification['scores'])
    if total_score != 0:
        for score in classification['scores']:
            score['score'] = score['score']/total_score
    return classification

### Função principal

* Divide o dataset em classificados e não classificados.
* Divide os classificados para teste e treino: 20% teste e 80% treino.
* Calcula em 50 interações a média da acurácia do modelo.

In [14]:
import numpy
import random

classified_df = []
unclassified_df = []

for data in df:
    classified_df.append(data) if data['is_classified'] else unclassified_df.append(data)

accuracies = []
iterations = 50
for i in range(iterations):
    train_dataset = []
    test_dataset = []
    random.shuffle(classified_df)
    train_dataset, test_dataset = np.split(classified_df, [int(len(classified_df)*0.8)])
    
    corpus = learning(train_dataset)
    classifications = [{'test_data': test_data, 'scores': classificate(corpus, test_data['text'])} for test_data in test_dataset]
    classifications = [normalize_scores(classification) for classification in classifications]
    
    right_answers = 0
    for classification in classifications:
        higher_score = max(classification['scores'], key=lambda x:x['score'])
        if classification['test_data']['class_name'] == higher_score['class_name']:
            right_answers += 1
    
    accuracy = right_answers/len(classifications)
    accuracies.append(accuracy)
    print(f'Iteration {i} - Accuracy {accuracy}')

print('-----------------')
print(f'Final accuracy {sum(accuracies)/len(accuracies)}')
print('-----------------')

Iteration 0 - Accuracy 0.5454545454545454
Iteration 1 - Accuracy 0.6233766233766234
Iteration 2 - Accuracy 0.5324675324675324
Iteration 3 - Accuracy 0.6363636363636364
Iteration 4 - Accuracy 0.6233766233766234
Iteration 5 - Accuracy 0.5194805194805194
Iteration 6 - Accuracy 0.5194805194805194
Iteration 7 - Accuracy 0.4675324675324675
Iteration 8 - Accuracy 0.5194805194805194
Iteration 9 - Accuracy 0.6363636363636364
Iteration 10 - Accuracy 0.6233766233766234
Iteration 11 - Accuracy 0.6103896103896104
Iteration 12 - Accuracy 0.5714285714285714
Iteration 13 - Accuracy 0.5844155844155844
Iteration 14 - Accuracy 0.5584415584415584
Iteration 15 - Accuracy 0.5584415584415584
Iteration 16 - Accuracy 0.5454545454545454
Iteration 17 - Accuracy 0.4935064935064935
Iteration 18 - Accuracy 0.6103896103896104
Iteration 19 - Accuracy 0.6493506493506493
Iteration 20 - Accuracy 0.4935064935064935
Iteration 21 - Accuracy 0.5714285714285714
Iteration 22 - Accuracy 0.5974025974025974
Iteration 23 - Accura